#Chapter 12

Markov assumption: n-gram

Chain Rule

P(how old are you)

= p(how)*p(old|how)*p(are|how,old) * p*(you|how,old,are)

this can be very slow if you have long sentances

k=1 (bi-gram)


p(are|how,old) = p(are|old)

k=2 (tri-gram)

p*(you|how,old,are) = p*(you|old,are)

k-1 (k-gram)


In [1]:
import spacy
import numpy as np
import re
nlp = spacy.load('en_core_web_sm')

def nGrams(doc_string_or_list,n,docIsAlreadyTokenized):
    doc = doc_string_or_list
    if not docIsAlreadyTokenized:
        doc = [token.text for token in nlp(doc_string_or_list)]
    doc = ' '.join(doc).lower().split(' ')
    grams = [doc[i:i+n] for i in range(len(doc)-n+1)]
    return grams

In [2]:
doc = 'how old are you today or can you tell me something about you'
n = 4
grams = nGrams(doc,n,False)
print(grams)

[['how', 'old', 'are', 'you'], ['old', 'are', 'you', 'today'], ['are', 'you', 'today', 'or'], ['you', 'today', 'or', 'can'], ['today', 'or', 'can', 'you'], ['or', 'can', 'you', 'tell'], ['can', 'you', 'tell', 'me'], ['you', 'tell', 'me', 'something'], ['tell', 'me', 'something', 'about'], ['me', 'something', 'about', 'you']]


In [3]:
from collections import defaultdict

def buildModel():
    model = defaultdict(lambda: defaultdict(lambda: 0))
    return model

def updateCount(nGram,model):
    w_1_to_n_minus_1 = tuple(nGram[:-1])
    w_n = nGram[-1]
    model[w_1_to_n_minus_1][w_n]+=1
    return model

def computeProbability(model):
    for w_1_to_n_minus_1 in model:
        totalCount = float(sum(model[w_1_to_n_minus_1].values()))
        for w_n in model[w_1_to_n_minus_1]:
            model[w_1_to_n_minus_1][w_n] /= totalCount
    return model

import dill

def saveModel(model,fileNameDotpkl):
    with open(fileNameDotpkl,'wb') as f:
        dill.dump(model,f)

def loadModel(fileNameDotpkl):
    with open(fileNameDotpkl,'rb') as f:
        model = dill.load(f)
    return model


from sklearn.datasets import fetch_20newsgroups as getData
from nltk.corpus import reuters as corpus

In [4]:
X,y = getData(subset='train',remove=('headers','footers','qoutes'),return_X_y=True)
print(X[0])
print(y[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.
7


In [5]:
import nltk
nltk.download('reuters')
nltk.download('punkt')

len(X)
n = 3
model = buildModel()

for doc in X:
    for nGram in nGrams(doc,n,False):
        model = updateCount(nGram,model)


for sentence in corpus.sents():
    for nGram in nGrams(sentence,n,True):
        model = updateCount(nGram,model)

model = computeProbability(model)

saveModel(model,'model.pkl')

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\lukem\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lukem\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [7]:
text = ['after','that']
nextWords = list(model[tuple(text[-n+1:])].keys())
probs = list(model[tuple(text[-n+1:])].values())
nextWord = np.random.choice(nextWords,1,probs)[0]
print(nextWord)

def sampleText(model,startingText=['after','that'],mxLenght = 100,nGramSize=3):
    text = startingText
    n = nGramSize
    while not len(text)>mxLenght:
        nextWords = list(model[tuple(text[-n+1:])].keys())
        probs = list(model[tuple(text[-n+1:])].values())
        if len(nextWords) > 0:
            nextWord = np.random.choice(nextWords,1,probs)[0]
            text.append(nextWord)
        else:
            break
    sampled = ' '.join(text)
    return sampled

for sents in nlp(sampleText(model,['due','to'])).sents:
    print(sents)
    break

date
due to continued expansion of eep wheat to eight billion marks liquidity , bills maturing from may to boost latin output latin american 
	 baseball players _ ; it was ordered dismantled by the fall compares with today ' s subsidy bill .
